# EDA en los datos crudos 

En el presente notebooks se hace una exploracion de los datos crudos para entender los tipos de datos y las problemáticas que podrian surgir en un proceso ETL con estos datos en el origen. 

In [10]:
# Importamos las librerías necesarias 

# Manejo de datos y exploración
import pandas as pd
import numpy as np

# Libreria auxiliar
from src.etl import *


In [18]:
# parametros 

DATA_RAW_DIRECTORY = "./data/raw/"
DATA_PREP_DIRECTORY = "./data/prep/"
DATA_CLEAN_DIRECTORY = "./data/clean/"

ORIGIN_FILE = "ventas_clientes.xlsx"
FINAL_FILE = "ventas_clientes.csv"

### Exploración inicial 

Probamos si el archivo tiene problemas de carga, de no ser asi, lo cargamos a memoria 

In [12]:
# Validamos la carga correcta 
cargar_data_excel(DATA_RAW_DIRECTORY + ORIGIN_FILE)

,Fecha,Region,Producto,Ventas,Clientes
0,2023-01-01,Norte,Producto A,210821,1689
1,2023-01-01,Norte,Producto B,531578,1269
2,2023-01-01,Norte,Producto C,158817,1678
3,2023-01-01,Norte,Producto D,455092,4465
4,2023-01-01,Norte,Producto E,450756,2213


In [13]:
# Guardamos en memoria los datos 
df = pd.read_excel(DATA_RAW_DIRECTORY + ORIGIN_FILE)

In [26]:
# Generamos un resumen de las columnas
res_col_df = resumen_columnas(df)
res_col_df["pc_nulos"] = (res_col_df["nulos"]/res_col_df["total"]).round(2)
res_col_df

,tipo_dato,valores_unicos,nulos,total,pc_nulos
Fecha,datetime64[ns],24,0,480,0.0
Region,object,4,0,480,0.0
Producto,object,5,0,480,0.0
Ventas,int64,479,0,480,0.0
Clientes,int64,459,0,480,0.0


In [27]:
res_col_df.reset_index().rename(columns = {"index":"Campo"}).to_csv(DATA_CLEAN_DIRECTORY + "resumen_datos.csv", index = False)

A partir de esto definimos el esquema de datos

In [15]:
schema_str = generar_schema_spark(res_col_df.reset_index().rename(columns = {"index":"columna", "tipo_dato":"tipo"})[["columna", "tipo"]])
print("El esquema de Spark recomendado es: ----------- \n", schema_str)

El esquema de Spark recomendado es: ----------- 
 StructType([
    StructField('Fecha', TimestampType(), True),
    StructField('Region', StringType(), True),
    StructField('Producto', StringType(), True),
    StructField('Ventas', IntegerType(), True),
    StructField('Clientes', IntegerType(), True)
])


### Pruebas de calidad y Limpieza

Realizamos pruebas de calidad y guardamos el archivo limpio

In [16]:
# Pruebas para Fecha
run_datetime_quality_tests(df, 'Fecha')

# Pruebas para tipos object
run_object_quality_tests(df, 'Region')
run_object_quality_tests(df, 'Producto')

# Pruebas para tipos int64
run_integer_quality_tests(df, 'Ventas')
run_integer_quality_tests(df, 'Clientes')

Transformaciones 

In [8]:
# Formato a columnas object
df = formatear_columnas_string(df, columnas=None)

# Fill NAs en entero 
df = rellenar_nulos_numericos(df, columna = "Ventas", metodo = 'mediana')

# Formateamos los nombres de las columnas
df.columns = [x.replace(" ","_").lower() for x in df.columns.tolist()]

In [9]:
# Guardamos el archivo , aún crudo, en fromato csv
guardar_data_csv(df, file_path = DATA_PREP_DIRECTORY + FINAL_FILE, index= False)

Archivo guardado exitosamente en: ./data/prep/ventas_clientes.csv
